# Start JupyterLab

To get the Jupyter kernel to work, run the following command in the terminal:

```bash
pixi run jupyter lab --no-browser
```
Then find the portion that reads:

```bash
Or copy and paste one of these URLs:
    http://localhost:8888/lab
    http://127.0.0.1:8888/lab
```

Copy and paste the URL into the spot that shows up at "Existing Jupyter Server" menu under "Select Kernel" button at the top right.

# Imports

In [1]:
from imgtools.io import read_dicom_series
from imgtools.autopipeline import AutoPipeline
import os
import pandas as pd
from readii.loaders import loadSegmentation
from readii.feature_extraction import singleRadiomicFeatureExtraction

from workflow.scripts.python.get_roi_names_with_count import getRTStructsRoiNames

# File path setup

This is an example with a CT scan and RTSTRUCT. Copy the below cell and replace with MRI data paths. Keep variable names the same.

In [2]:
# Path to the DIRECTORY that contains all the CT dcm files
image_dir_path = "rawdata/4D-Lung/113_HM10395/11-26-1999-NA-p4-13296/1.000000-P4P113S303I10349 Gated 40.0B-29543/"

# Path to the RTSTRUCT FILE
rtstruct_file_path = "rawdata/4D-Lung/113_HM10395/11-26-1999-NA-p4-13296/1.000000-P4P113S303I10349 Gated 40.0B-47.35/1-1.dcm"

# Run Med-Imagetools crawl

This creates a file used by READII to associate the images with the corresponding segmentation files.

In [3]:
# CHANGE THESE THREE PATHS TO ISPY
mit_input_dir = "rawdata/4D-Lung"
mit_output_dir = "procdata/4D-Lung/mit_output"
# CHANGE CT TO MR
modalities = "CT,RTSTRUCT"
# LEAVE THIS ONE
dry_run = False

# Set up and run the pipeline
pipeline = AutoPipeline(input_directory=mit_input_dir,
                        output_directory=mit_output_dir,
                        modalities=modalities,
                        dry_run=dry_run,
                        show_progress=False)

pipeline.run()

The dataset has already been indexed.
Edge table is already present. Loading the data...
Forming the graph based on the given modalities: CT,RTSTRUCT
There are 1 cases containing all CT,RTSTRUCT modalities.
Dataset already processed...


/Users/katyscott/Documents/Radiogenomics/I-SPY2/ispy-readii/.pixi/envs/default/lib/python3.12/site-packages/imgtools/modules/datagraph.py:428: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  relevant_study_id = self.df_new.loc[(self.df_new.edge_type.str.contains(regex_term)), "study_x"].unique()


# List ROI names in the RTSTRUCT file

To load in the RTSTRUCT and get just the ROI we want, we need to make a dictionary with all the possible variations of the ROI names.

E.g. for a GTV ROI, we might have: GTV, GTVp, GTV1, GTP-p, etc.

Using this will rename any ROI that matches one of these names to GTV.

In [4]:
# List all the ROI names present in the RTSTRUCT 
getRTStructsRoiNames(mit_input_dir)

{'Vertebra_c40': 1, 'Carina_c40': 1, 'LN_c40': 1, 'Tumor_c40': 1}

In [5]:
# Select the name you want to give to your ROIs (e.g. GTV or tumor) and list any ROI names from previous cell output you want to include
roi_name = "GTV"
roi_name_options = {roi_name: ["Tumor_c40"]}

# Load in image and segmentation

In [6]:
# Load the image (CT or MR)
image = read_dicom_series(image_dir_path)

# Load in the segmentation file (this is set up for RTSTRUCT)
# This will return a dictionary of segmentation images, with the keys being the ROI names
seg_image_dict = loadSegmentation(rtstruct_file_path, modality = 'RTSTRUCT', 
                                  baseImageDirPath=image_dir_path, roiNames=roi_name_options)

# Get the segmentation image for the ROI of interest
seg_image = seg_image_dict[roi_name]

2024-11-13T15:39:16-0500 [debug    ] Loading RTSTRUCT               call=loaders.loadRTSTRUCTSITK:74
2024-11-13T15:39:16-0500 [debug    ] Creating StructureSetToSegmentation object call=loaders.loadRTSTRUCTSITK:79 continuous=True roiNames={'GTV': ['Tumor_c40']}
2024-11-13T15:39:16-0500 [debug    ] Finished making mask           call=loaders.loadRTSTRUCTSITK:99 seg_roi_names={'GTV': ['Tumor_c40']}


labels: {'GTV': ['Tumor_c40']}


# Confirm that loading working correctly

The dimensions of the image and segmenation should match.

In [7]:
print("Image dimensions: ", image.GetSize())
print("Segmentation dimensions: ", seg_image.GetSize())

Image dimensions:  (512, 512, 99)
Segmentation dimensions:  (512, 512, 99)


## Extract Radiomic Features

In [8]:
# Pick which config settings to use for PyRadiomics feature extraction
pyradiomics_param_file_path = "workflow/config/default_pyradiomics.yaml"

# Run PyRadiomics feature extraction with REAII function
radiomic_features_dict = singleRadiomicFeatureExtraction(image, seg_image, pyradiomics_param_file_path, randomSeed=10)

# Convert to dataframe
rad_features_df = pd.DataFrame.from_dict(radiomic_features_dict, orient="index").transpose()


2024-11-13T15:39:35-0500 [info     ] Setting up Pyradiomics feature extractor... call=feature_extraction.singleRadiomicFeatureExtraction:123
2024-11-13T15:39:35-0500 [info     ] Starting radiomic feature extraction... call=feature_extraction.singleRadiomicFeatureExtraction:130


# Save out the results of the feature extraction

In [9]:
save_radiomics_path = "results/4D-Lung/readii_radiomic_features.csv"

if not os.path.exists(os.path.dirname(save_radiomics_path)):
    os.makedirs(os.path.dirname(save_radiomics_path))

rad_features_df.to_csv(save_radiomics_path, index=False)